In [1]:
import SimPEG as simpeg
import simpegMT as simpegmt
import numpy as np

In [2]:
## Setup the problem

# Frequency
nFreq = 33
# freqs = np.logspace(3,-3,nFreq)
freqs = np.array([100,10,1,0.1,0.01])
# Make the mesh
ct = 5
air = simpeg.Utils.meshTensor([(ct,25,1.3)])
core = np.concatenate( (  np.kron(simpeg.Utils.meshTensor([(ct,15,-1.2)]),np.ones((10,))) , simpeg.Utils.meshTensor([(ct,20)]) ) )
bot = simpeg.Utils.meshTensor([(core[0],10,-1.3)])
x0 = -np.array([np.sum(np.concatenate((core,bot)))])
# Change to use no air
m1d = simpeg.Mesh.TensorMesh([np.concatenate((bot,core,air))], x0=x0)

## Setup model varibles
active = m1d.vectorCCx<0.
layer = (m1d.vectorCCx<-200.) & (m1d.vectorCCx>=-600.)
actMap = simpeg.Maps.ActiveCells(m1d, active, np.log(1e-8), nC=m1d.nCx)
mappingExpAct = simpeg.Maps.ExpMap(m1d) * actMap
sig_half = 2e-3
sig_air = 1e-8
sig_layer = 1e-3
# Make the true model
sigma_true = np.ones(m1d.nCx)*sig_air
sigma_true[active] = sig_half
sigma_true[layer] = sig_layer
m_true = np.log(sigma_true[active])
# Make the background model
sigma_0 = np.ones(m_true.size)*sig_half
m_0 = np.log10(np.ones(m_true.size)*sig_half)

# Receivers 
# 1D impedance at the surface (elevation 0)
rxList = []
for rxType in ['z1dr','z1di']:
    rxList.append(simpegmt.SurveyMT.RxMT(simpeg.mkvc(np.array([0.0]),2).T,rxType))
# Source list
srcList =[]
tD = False
if tD:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1DhomotD(rxList,freq))
else:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1Dprimary(rxList,freq,sigma_0))
# Make the survey
survey = simpegmt.SurveyMT.SurveyMT(srcList)
survey.mtrue = sigma_true
# Set the problem
problem = simpegmt.ProblemMT1D.eForm_psField(m1d,mapping=mappingExpAct)
problem.pair(survey)

In [3]:
np.sum(active)

180

In [4]:
## Make the observed data 
# Project the data
d_true = survey.dpred(m_true)
survey.dtrue = d_true
# Add noise
std = 0.05 # 5% std
noise = std*abs(survey.dtrue)*np.random.randn(*survey.dtrue.shape)
# Assign the dobs
survey.dobs = survey.dtrue + noise
survey.std = survey.dobs*0 + std
# Assign the data weight
survey.Wd = 1/(abs(survey.dobs)*std)

Exception: Unexpected shape of tensor

In [ ]:
## Setup the inversion proceedure
C =  simpeg.Utils.Counter()

# Set the optimization
opt = simpeg.Optimization.InexactGaussNewton(maxIter = 10)
opt.counter = C
opt.LSshorten = 0.5
opt.remember('xc')
# Data misfit
dmis = simpeg.DataMisfit.l2_DataMisfit(survey)
# Regularization
reg = simpeg.Regularization.Tikhonov(m1d)
reg.alpha_s = 1e-5
reg.alpha_x = 1.

# Inversion problem
invProb = simpeg.InvProblem.BaseInvProblem(dmis, reg, opt)
invProb.counter = C
# Beta cooling
beta = simpeg.Directives.BetaSchedule()
betaest = simpeg.Directives.BetaEstimate_ByEig(beta0_ratio=0.75)
saveModel = simpeg.Directives.SaveModelEveryIteration()
# Create an inversion object
inv = simpeg.Inversion.BaseInversion(invProb, directiveList=[beta,betaest,saveModel]) 


In [ ]:
# Runn the inversion
mopt = inv.run(m_0)

In [ ]:
opt.counter.summary()
xc = opt.recall('xc')

In [ ]:
import matplotlib.pyplot as plt
# plt.figure(1)
# for i in range(problem.G.shape[0]):
#     plt.plot(problem.G[i,:])

plt.figure(2)
plt.plot(m1d.vectorCCx, np.log10(survey.mtrue), 'b-')
plt.plot(m1d.vectorCCx, np.log10(mopt), 'r-')
plt.show()


In [ ]:
mopt